# Deploy multiple machine learning models from registry to online endpoint  

Learn how to use an online endpoint to deploy your model, so you don't have to create and manage the underlying infrastructure. You'll begin by deploying a model on your local machine to debug any errors, and then you'll deploy and test it in Azure.

Managed online endpoints help to deploy your ML models in a turnkey manner. Managed online endpoints work with powerful CPU and GPU machines in Azure in a scalable, fully managed way. Managed online endpoints take care of serving, scaling, securing, and monitoring your models, freeing you from the overhead of setting up and managing the underlying infrastructure. 

For more information, see [What are Azure Machine Learning endpoints?](https://docs.microsoft.com/azure/machine-learning/concept-endpoints).

## Prerequisites

* Install and configure the [Python SDK v2](sdk/setup.sh).

* You must have an Azure resource group, and you (or the service principal you use) must have Contributor access to it.

* You must have an Azure Machine Learning workspace. 

* Train Iris model and register multiple versions [Iris Training](./train-iris-scikit-learn.ipynb)

## Steps

* Connect to Workspace

* Create Endpoint

* Add Role 'AzureML DataScientist'  to Endpoint Managed Identity one Workspace scope 

* Create Deployment with scoring scipt

* Scoring Scipt Loads  multiple models from WS  Registry 

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [2]:

# Enter details of your AML workspace
subscription_id = "SUB"
resource_group = "RG"
workspace_name = "WS"


user_identity_id="/subscriptions/...../providers/Microsoft.ManagedIdentity/userAssignedIdentities/..."


In [3]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)

# 4. Deploy your online endpoint to Azure
Next, deploy your online endpoint to Azure.



## 4.1 Create  Managed Endpoint Identity and add Role
To access ML Models from Registry in workspace Managed Online Endpoint System Identity needs to be granted access to registry.
Assign `AzureML Data Scientist` role to MOE Identity on Workspace scope
![identity](docs/modelidentity.png)

- Reader on Workspace
- ACRPull on ACR
- Blob Storage Reader on Storage

https://learn.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints?view=azureml-api-2&tabs=cli#authorization-error 

## 4.2 Configure online endpoint
`endpoint_name`: The name of the endpoint. It must be unique in the Azure region. Naming rules are defined under [managed online endpoint limits](https://docs.microsoft.com/azure/machine-learning/how-to-manage-quotas#azure-machine-learning-managed-online-endpoints-preview).

`auth_mode` : Use `key` for key-based authentication. Use `aml_token` for Azure Machine Learning token-based authentication. A `key` does not expire, but `aml_token` does expire. 

Optionally, you can add description, tags to your endpoint.

In [5]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime
from azure.ai.ml.entities import ManagedIdentityConfiguration, IdentityConfiguration

online_endpoint_name = "multimodel-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a multimodel online endpoint",
    auth_mode="key",
    identity=IdentityConfiguration(
        type="user_assigned",
        user_assigned_identities=[
            ManagedIdentityConfiguration(resource_id=user_identity_id)
        ],
    ),
    tags={"foo": "bar"},
)

## 4.2.1 Create the endpoint

Using the `MLClient` created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [6]:
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://multimodel-05100259870723.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://multimodel-05100259870723.eastus2.inference.ml.azure.com/swagger.json', 'name': 'multimodel-05100259870723', 'description': 'this is a multimodel online endpoint', 'tags': {'foo': 'bar'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/f9b97038-ed78-4a26-a1a7-51e81e75d867/resourcegroups/openaml/providers/microsoft.machinelearningservices/workspaces/nlp-workspace/onlineendpoints/multimodel-05100259870723', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/f9b97038-ed78-4a26-a1a7-51e81e75d867/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oe:baa4dabf-18ba-45e2-8649-6d72d7082169:83f1d8e6-1b46-4e57-9da8-94c0d3d0d39a?api-version=2022-02-01-preview'}, 'id': '/subscriptions/f9b97038-ed78-4a26-a1a7-51e81e75d867/resourceGroups

In [7]:
endpoint = ml_client.online_endpoints.get(online_endpoint_name)
print("Endpoint Identity {0} ".format(endpoint.identity))

Endpoint Identity <azure.ai.ml.entities._credentials.IdentityConfiguration object at 0x7fa369ff8f10> 


## 4.3 Configure online deployment



## 4.3.1 Explore Scoring script

Scoring script is in `onlinescoring/score_registry.py` reads MLFlow Tracking URI from environment variable  (set during registration) and loads models from workspace for prediction

```python
import os
import logging
import json
import numpy
import joblib
import mlflow


def init():
    """
    This function is called when the container is initialized/started, typically after create/update of the deployment.
    You can write the logic here to perform init operations like caching the model in memory
    """
    global model1, model2
    # More details  https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-models-mlflow
    mlflow.set_tracking_uri(os.getenv("TRACKING_URI"))
    logging.info("Tracking URL: {0}".format(os.getenv("TRACKING_URI")))

    model1 = mlflow.sklearn.load_model("models:/iris_svc_model/1")
    model2 = mlflow.sklearn.load_model("models:/iris_svc_model/latest")
    logging.info("Init complete")


def run(raw_data):
    """
    This function is called for every invocation of the endpoint to perform the actual scoring/prediction.
    In the example we extract the data from the json input and call the scikit-learn model's predict()
    method and return the result back
    """
    logging.info("Request received")
    json_data = json.loads(raw_data)
    if "input_data" not in json_data.keys():
        raise Exception("Request must contain a top level key named 'input_data'")
  
    
    input_data = json_data["input_data"]
    print(json.dumps(input_data))

    data = input_data["data"]
    data = numpy.array(data)
    num_rows, num_cols = data.shape # just example based on number of rows to switch model

    if ( num_rows <= 2 ) :
      logging.info(" Model 1 is predicting")
      result = model1.predict(data)
    else:
      logging.info(" Model 2 is predicting")
      result = model2.predict(data)  
    logging.info("Request processed")
    return result.tolist()

```

## 4.3.2 Register Deployment
A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `ManagedOnlineDeployment` class.

In [17]:
env = Environment(
    conda_file="./environment/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
workspace = ml_client.workspaces.get(name=workspace_name)

blue_deployment = ManagedOnlineDeployment(
    name="blue" + datetime.datetime.now().strftime("%m%d%H%M%f"),
    endpoint_name=online_endpoint_name,
    #model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="./onlinescoring", scoring_script="score_registry_uai.py"
    ),
    environment_variables={
        "TRACKING_URI": workspace.mlflow_tracking_uri,
        "UAI_CLIENT_ID": user_identity_id
    },
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

print("Deployment {0}  defined".format(blue_deployment.name))


Deployment blue05100417692517  defined


## 4.4 Create the deployment

Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [18]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint multimodel-05100259870723 exists
Uploading onlinescoring (0.0 MBs): 100%|██████████| 2891/2891 [00:00<00:00, 49612.87it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


................................................................

HttpResponseError: (None) ResourceNotReady: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready
Code: None
Message: ResourceNotReady: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready
Exception Details:	(None) ResourceNotReady: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready
	The build log is available in the workspace blob store "nlpworkspace8557870865" under the path "/azureml/ImageLogs/11ded7f8-0d35-422e-bf2a-33f3547358da/build.log"
	Code: None
	Message: ResourceNotReady: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready
	The build log is available in the workspace blob store "nlpworkspace8557870865" under the path "/azureml/ImageLogs/11ded7

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {f"{blue_deployment.name}": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

# 5. Test the endpoint with sample data
Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the `invoke` command with the following parameters:
- `endpoint_name` - Name of the endpoint
- `request_file` - File with request data
- `deployment_name` - Name of the specific deployment to test in an endpoint

We will send a sample request using a [json](./model-1/sample-request.json) file. 

In [ ]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=blue_deployment.name,
    request_file="./onlinescoring/sample-request.json",
)

# 6. Managing endpoints and deployments

## 6.1 Get details of the endpoint

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

## 6.2 Get the logs for the new deployment
Get the logs for the green deployment and verify as needed

In [ ]:
ml_client.online_deployments.get_logs(
    name=blue_deployment.name, endpoint_name=online_endpoint_name, lines=50
)

# 7. Delete the endpoint


In [ ]:
#ml_client.online_endpoints.begin_delete(name=online_endpoint_name)